# 5. __Classical Controllers__

- In practicality an energy network is always in flux. 
- The stochastic nature of the the loads and renewable generation can be simulated. 

## Ornstein–Uhlenbeck process

>- This example will show you how to add a stochastic reference signal to real power set points.
>- The Ornstein–Uhlenbeck process can be thought of as a relaxation process with Brownian noise added on.
>- The functionality to accept stochastic reference signals is available for both VSG and PQ sources.
>- All the typical parameters for a OU process is available
>>- If not specified the asymptotic long term mean,γ, is set equal to the real power set point.
>>- The power factor is kept constant.
>>- The mean reversion parameter, κ (kappa), is automatically calculated from the asymptotic standard deviation. 
>- Two additional stochastic parameters are available
>>- The process time step, Δt, indicates the interval until a new random value is generated.
>>- Newton interpolation is used to smooth the process, which can be varied with interpolation degree, k. Values larger than 2 are not advised. Only integer values are accepted.

The Ornstein-Uhlenbeck process is described by the following SDE:

$$
  dX_t = κ(γ-X_t)dt + σdW_t
$$



In [1]:
using JEG;

WebIO._IJuliaInit()

_______________________________________________________________________________
### Network Configuration 

>- Two inverters, a load, and two cables.
>- The one inverter is placed in the familiar PQ mode, while the other is in VSG mode.
>- The simulation is run for multiple episodes.


In [2]:
t_end    = 0.2     # total run time, seconds
num_eps  = 4       # number of episodes to run   

# Connectivity Matrix
CM = [ 0. 0. 1.
        0. 0. 2.
        -1. -2. 0.]  

# Nominal 3-phase power rating [VA], power factor, nominal rms voltage
R_load, L_load, _, _ = ParallelLoadImpedance(10e3, 0.95, 230) 

parameters = Dict{Any, Any}(
        "source" => Any[
                        Dict{Any, Any}("pwr"       => 200e3,
                                        "mode"     => "VSG")
                        Dict{Any, Any}("pwr"       => 100e3,   
                                        "mode"     => "PQ",    
                                        "p_set"    => -30e3, # Asymptotoic Mean
                                        "q_set"    => 10e3,
                                        "std_asy"  => 2.5e3, # Asymptotic Standard Deviation
                                        "σ"        => 100e3, # Brownian motion scale
                                        "Δt"       => 0.01,  # Process time Step, seconds
                                        "X₀"       => 0.0,   # Initial Process Value, Watt
                                        "k"        => 0)     # Interpolation degree, {0, 1, 2}
                        ],
        "load"   => Any[
                        Dict{Any, Any}("impedance" => "RL", 
                                        "R"        => R_load, 
                                        "L"        => L_load),
                        ],
        "cable"   => Any[
                        Dict{Any, Any}("R"  => 0.1, 
                                        "L" => 0.25e-3, 
                                        "C" => 0.1e-4),
                        Dict{Any, Any}("R"  => 0.1, 
                                        "L" => 0.25e-3, 
                                        "C" => 0.1e-4),
                        ],
    );

env = ElectricGridEnv(CM = CM, parameters = parameters, t_end = t_end, verbosity = 2);

agents = SetupAgents(env);

hook = Simulate(agents, env, num_episodes = num_eps, );

[ Info: Normalization is done based on the defined parameter limits.
[ Info: Time simulation run time: 0.2 [s] ~> 2001 steps
[ Info: 2 'classically' controlled sources have been initialised.
[ Info: 1 source has been set up in PQ mode.
[ Info: 1 source has been set up in Synchronverter mode.
[ Info: All 'classically' controlled sources have been automatically set up with droop coeficients, and proportional and integral gains.
[ Info: 1 stochastic process will start at 0.04 [s].
Progress: 100%|█████████████████████████████████████████| Time: 0:00:38


_______________________________________________________________________________
### Rendering

In [3]:
RenderHookResults(hook = hook, 
                    episode = 2,
                    states_to_plot  = [], 
                    actions_to_plot = [],  
                    power_p_inv     = [2],   # Real power [Watts]
                    power_q_inv     = [2],   # Imaginary power [VAi]
                    )

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields height, legend, margin, plot_bgcolor, template, width, xaxis, yaxis, and yaxis2"

_______________________________________________________________________________
### Analysis

- The real and imaginary powers one the PQ load fluctuate around their set points. 